## ⚽ Football Object Detection using YOLOv11-Large

This model is designed to detect key entities on a football field:  
- 🧤 **Goalkeeper**  
- 🧍 **Player**  
- ⚪ **Ball**  
- 🧑‍⚖️ **Referee**

Built on the powerful **YOLOv11-Large** architecture with transfer learning, the first half of the layers were frozen to retain general features while fine-tuning the rest on football-specific data.

---

### 🔧 Configuration

- **Architecture**: YOLOv11-Large
- **Frozen Layers**: First half
- **Pre-trained Weights**: Used
- **Epochs Trained**: 50
- **Learning Rate**: ~3.73e-5 (pg0/pg1/pg2)
- **Train Duration**: 539 minutes
- **Model Size**: 25.3M parameters, 87.3 GFLOPs

---

### 📊 Evaluation Metrics

> *Values are reported as: Final / Best during training*

| Metric                     | Final      | Best (Min)    |
|----------------------------|------------|---------------|
| **mAP@50-95 (val)**        | 0.53167    | 0.53167       |
| **mAP@50 (val)**           | 0.81680    | 0.81737       |
| **Precision (val)**        | 0.90306    | 0.91265       |
| **Recall (val)**           | 0.74100    | 0.78776       |
| **Box Loss (train)**       | 0.84756    | 0.83256       |
| **Class Loss (train)**     | 0.41376    | 0.40873       |
| **DFL Loss (train)**       | 0.80578    | 0.80386       |
| **Box Loss (val)**         | 0.86958    | 0.86958       |
| **Class Loss (val)**       | 0.42202    | 0.42202       |
| **DFL Loss (val)**         | 0.81992    | 0.81951       |

---

### 🧪 Notes

- The model converged smoothly with no signs of overfitting.
- High **precision (0.91)** and solid **recall (0.78)** suggest strong localization and classification.
- Model complexity (87.3 GFLOPs) makes it suitable for powerful edge or server-side deployment.

---

### 🚀 Potential Use Cases

- **Real-time football analytics**
- **Referee decision assist systems**
- **Player movement tracking & ball possession**
- **Broadcast visuals enhancement**



### Installing Dependencies

In [ ]:
# !pip3 install -qU torch torchvision torchaudio 
%pip install -qU comet_ml
%pip install -qU ultralytics

### Importing Dependency

In [ ]:
import comet_ml
import torch
from ultralytics import YOLO
import urllib.request as request
import os
from pathlib import Path
import zipfile


## Preparing Dataset
- You can use the same dataset from kaggle as well which is also shown in in input section

In [ ]:
# #Model foe getting the size of dataset

# def get_size(path: Path) -> str:
#     """get size in KB

#     Args:
#         path (Path): path of the file

#     Returns:
#         str: size in KB
#     """
#     size_in_kb = round(os.path.getsize(path)/1024)
#     return f"~ {size_in_kb} KB"

In [ ]:

# data_url = r"https://github.com/itsmeAman03/My_datasets/raw/refs/heads/main/Football-Players-Detection-Dataset.zip"
# data_name = "football-obd.zip"
# unzip_dir = "football-obd"

In [ ]:
#Data Download function
# def download_file(data_url,data_name):
#     if not os.path.exists(data_name):
#         filename, headers = request.urlretrieve(
#             url = data_url,
#             filename = data_name
#         )
#         print(f"{filename} download! with following info: \n{headers}")
#     else:
#       print(f"File already exists of size: {get_size(Path(data_name))}")

In [ ]:
# #Data Extract function
# def extract_zip_file(unzip_dir,data_name):
#     """
#     zip_file_path: str
#     Extracts the zip file into the data directory
#     Function returns None
#     """
#     unzip_path = Path(unzip_dir)
#     os.makedirs(unzip_path, exist_ok=True)
#     with zipfile.ZipFile(data_name, 'r') as zip_ref:
#       zip_ref.extractall(unzip_path)


In [ ]:
# STAGE_NAME = "Data Ingestion Stage"

# try:
#   print(f">>>>>> Stage {STAGE_NAME} Started <<<<<<")
#   download_file(data_url,data_name)
#   extract_zip_file(unzip_dir,data_name)
#   print(f">>>>>> Stage {STAGE_NAME} Completed <<<<<<\n\nX-----------------X")
# except Exception as e:
#   print("f An error occurred while executing the Data Ingest stage.\n {e}")

In [ ]:
#Loading the model
model = YOLO("/kaggle/input/yolo11/pytorch/default/1/yolo11l.pt")

In [ ]:
# Freeze the first N layers of the model (by module, not by parameter order)

modules = list(model.model.children())  # model.model is usually nn.Sequential in YOLO
print(f"Total modules in model.model: {len(modules)}")

N = 2*(len(modules)//3) # Set N to the number of layers you want to freeze

for i, module in enumerate(modules):
    if i < N:
        for param in module.parameters():
            param.requires_grad = False
        print(f"Froze module {i}: {module.__class__.__name__}")
    else:
        for param in module.parameters():
            param.requires_grad = True

# Check how many parameters are trainable
trainable = sum(p.requires_grad for p in model.parameters())
total = sum(1 for _ in model.parameters())
print(f"Trainable parameters: {trainable}/{total}")

In [ ]:
total_gpu = torch.cuda.device_count()
print(f"Number of GPUs available: {total_gpu}")

## I have used CometML for my Model training logging and ultraritics provide the simple addition in the code to CometML Experimentation

In [ ]:
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# COMET_API_KEY = user_secrets.get_secret("COMET_API_KEY")

In [ ]:
# comet_ml.login(api_key=COMET_API_KEY,project_name="football-object-detection-yolo11")

In [ ]:
# Train the model with the half parameters unfrozen
#yolo selects the gpu automatically so no need to sent onto gpu
results = model.train(
    data='/kaggle/input/football-players-detection-dataset/data.yaml',  # Path to the data configuration file
    epochs=50,                  # Number of training epochs
    imgsz=640,                  # Image size
    batch=16,                   # Batch size
    name='yolov11l_finetune',   # Name of the training run
    project="football-object-detection-yolo11",
    save_json=True,
    plots=True,
    device=[0,1],
)

### I wanted to download my training output weights, so I converted into Zip

In [ ]:
nested_path = "/kaggle/working/Saved_Model/"
os.mkdir(nested_path)

In [ ]:

onnxPath = model.export(format="onnx")
torchScript_path = model.export(format="torchscript")

print(f"✅ Full model Checkpoints to {onnxPath}")
print(f"✅ Full model Checkpoints to {torchScript_path}")


In [ ]:
%cd /kaggle/working
!zip -r model_saved.zip football-object-detection-yolo11/

In [ ]:
from IPython.display import FileLink
FileLink(r'model_saved.zip')